# Import Dependencies

In [1]:
# Import our dependencies
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

# Load in CSVs

In [2]:
math_file_path = os.path.join("Student_alcohol_consumption", "student-mat.csv")
math = pd.read_csv(math_file_path)

por_file_path = os.path.join("Student_alcohol_consumption", "student-por.csv")
por = pd.read_csv(por_file_path)

# Preprocessing
###### Very little cleaning needed to be done. Only the grades needed to be bucketed into pass and fail. OneHotEncoder and StandardScaler could handle the rest.

In [3]:
# clean bucket the grades
# 10 and above is a pass
# 9 and below is a fail

def encode_grades(data):
    # bucket the grades into passing(1) and failling(0)
    
    # math
    # failling
    data.loc[(data["G1"] < 10), "G1"] = 0
    data.loc[(data["G2"] < 10), "G2"] = 0
    data.loc[(data["G3"] < 10), "G3"] = 0

    #passing
    data.loc[(data["G1"] >= 10), "G1"] = 1
    data.loc[(data["G2"] >= 10), "G2"] = 1
    data.loc[(data["G3"] >= 10), "G3"] = 1
    
    display(data)
    
    return

encode_grades(math)
encode_grades(por)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,0,0,0
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,0,0,0
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,0,0,1
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,1,1,1
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,0,0,0
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,1,1,1
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,1,0,0
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,1,1,1


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,1,1
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,0,1,1
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,1,1,1
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,1,1,1
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,R,GT3,T,2,3,services,other,...,5,4,2,1,2,5,4,1,1,1
645,MS,F,18,U,LE3,T,3,1,teacher,services,...,4,3,4,1,1,1,4,1,1,1
646,MS,F,18,U,GT3,T,1,1,other,other,...,1,1,1,1,1,5,6,1,1,0
647,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,6,1,1,1


# Deep Neural Networks
###### I tried and would of liked to make this look nicer by making a function to handle this so I would not have needed to just copy and paste it 6 times.

In [4]:
def dnn(data, target, drop, file_name):

    # Generate our categorical variable list
    encode_cat = data.dtypes[data.dtypes == "object"].index.tolist()

    # Check the number of unique values in each column
    data[encode_cat].nunique()

    # Create the OneHotEncoder instance
    enc = OneHotEncoder(sparse=False)

    # Fit the encoder and produce encoded DataFrame
    encode_df = pd.DataFrame(enc.fit_transform(data[encode_cat]))

    # Rename encoded columns
    encode_df.columns = enc.get_feature_names(encode_cat)
    #display(encode_df.head())

    # Merge the two DataFrames together and drop the Country column
    data = data.merge(encode_df,left_index=True,right_index=True).drop(encode_cat, 1)
    #display(math.head())

    # Split our preprocessed data into our features and target arrays
    y = data[target].values
    X = data.drop(drop,1).values

    # Split the preprocessed data into a training and testing dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

    # Create a StandardScaler instance
    scaler = StandardScaler()

    # Fit the StandardScaler
    X_scaler = scaler.fit(X_train)

    # Scale the data
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    # Define the model - deep neural net
    number_input_features = len(X_train[0])
    hidden_nodes_layer1 =  len(X_train[0]) * 2
    hidden_nodes_layer2 = len(X_train[0]) * .1

    nn = tf.keras.models.Sequential()

    # First hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

    # Second hidden layer
    nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

    # Output layer
    nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Check the structure of the model
    nn.summary()

    # Compile the model
    nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

    # Train the model
    fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

    # Evaluate the model using the test data
    model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

    # Create a DataFrame containing training history
    history_df = pd.DataFrame(fit_model.history, index=range(1,len(fit_model.history["loss"])+1))

    # Plot the loss
    #display(history_df.plot(y="loss"))

    # Plot the accuracy
    #display(history_df.plot(y="accuracy"))
    
    # example file name
    #nn.save("trained_por_g3.h5")
    
    # save model
    nn.save(file_name)
    
    return model_accuracy

In [5]:
G1 = ["G1", "G2", "G3"]
G2 = ["G2", "G3"]
G3 = ["G3"]

dnn(math, "G1", G1, "trained_math_G1.h5")
dnn(math, "G2", G2, "trained_math_G2.h5")
dnn(math, "G3", G3, "trained_math_G3.h5")

dnn(por, "G1", G1, "trained_por_G1.h5")
dnn(por, "G2", G2, "trained_por_G2.h5")
dnn(por, "G3", G3, "trained_por_G3.h5")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 112)               6384      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 565       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 6,955
Trainable params: 6,955
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
10/10 [==============================] - 0s 1000us/step - loss: 0.7340 - accuracy: 0.4088
Epoch 2/50
10/10 [==============================] - 0s 1000us/step - loss: 0.6861 - accuracy: 0.5169
Epoch 3/50
10/10 [==============================] - 0s 900us/step - loss: 0.6660 - accuracy: 0.6588
Epoch 4/50
10/10 [==============================] - 0s 800us/step - loss: 0.6511 -

Epoch 17/50
10/10 [==============================] - 0s 800us/step - loss: 0.1646 - accuracy: 0.9561
Epoch 18/50
10/10 [==============================] - 0s 1000us/step - loss: 0.1500 - accuracy: 0.9628
Epoch 19/50
10/10 [==============================] - 0s 1000us/step - loss: 0.1388 - accuracy: 0.9628
Epoch 20/50
10/10 [==============================] - 0s 1ms/step - loss: 0.1271 - accuracy: 0.9696
Epoch 21/50
10/10 [==============================] - 0s 1ms/step - loss: 0.1168 - accuracy: 0.9696
Epoch 22/50
10/10 [==============================] - 0s 1ms/step - loss: 0.1073 - accuracy: 0.9764
Epoch 23/50
10/10 [==============================] - 0s 999us/step - loss: 0.0996 - accuracy: 0.9764
Epoch 24/50
10/10 [==============================] - 0s 1000us/step - loss: 0.0921 - accuracy: 0.9797
Epoch 25/50
10/10 [==============================] - 0s 1ms/step - loss: 0.0855 - accuracy: 0.9797
Epoch 26/50
10/10 [==============================] - 0s 1000us/step - loss: 0.0800 - accuracy: 0

10/10 [==============================] - 0s 800us/step - loss: 0.0252 - accuracy: 0.9966
Epoch 39/50
10/10 [==============================] - 0s 1ms/step - loss: 0.0239 - accuracy: 0.9966
Epoch 40/50
10/10 [==============================] - 0s 1000us/step - loss: 0.0225 - accuracy: 0.9966
Epoch 41/50
10/10 [==============================] - 0s 1000us/step - loss: 0.0215 - accuracy: 0.9966
Epoch 42/50
10/10 [==============================] - 0s 1000us/step - loss: 0.0204 - accuracy: 0.9966
Epoch 43/50
10/10 [==============================] - 0s 1000us/step - loss: 0.0194 - accuracy: 0.9966
Epoch 44/50
10/10 [==============================] - 0s 1000us/step - loss: 0.0185 - accuracy: 0.9966
Epoch 45/50
10/10 [==============================] - 0s 900us/step - loss: 0.0172 - accuracy: 0.9966
Epoch 46/50
10/10 [==============================] - 0s 1000us/step - loss: 0.0161 - accuracy: 0.9966
Epoch 47/50
10/10 [==============================] - 0s 1000us/step - loss: 0.0149 - accuracy: 1.00

16/16 [==============================] - 0s 980us/step - loss: 0.6151 - accuracy: 0.7037
Epoch 2/50
16/16 [==============================] - 0s 1ms/step - loss: 0.4827 - accuracy: 0.7819
Epoch 3/50
16/16 [==============================] - 0s 1ms/step - loss: 0.4187 - accuracy: 0.8086
Epoch 4/50
16/16 [==============================] - 0s 812us/step - loss: 0.3765 - accuracy: 0.8148
Epoch 5/50
16/16 [==============================] - 0s 875us/step - loss: 0.3456 - accuracy: 0.8374
Epoch 6/50
16/16 [==============================] - 0s 875us/step - loss: 0.3182 - accuracy: 0.8457
Epoch 7/50
16/16 [==============================] - 0s 937us/step - loss: 0.2944 - accuracy: 0.8683
Epoch 8/50
16/16 [==============================] - 0s 937us/step - loss: 0.2728 - accuracy: 0.8765
Epoch 9/50
16/16 [==============================] - 0s 875us/step - loss: 0.2488 - accuracy: 0.9033
Epoch 10/50
16/16 [==============================] - 0s 937us/step - loss: 0.2306 - accuracy: 0.9115
Epoch 11/50
16

16/16 [==============================] - 0s 750us/step - loss: 0.0490 - accuracy: 0.9835
Epoch 20/50
16/16 [==============================] - 0s 1000us/step - loss: 0.0434 - accuracy: 0.9918
Epoch 21/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0384 - accuracy: 0.9938
Epoch 22/50
16/16 [==============================] - 0s 937us/step - loss: 0.0342 - accuracy: 0.9959
Epoch 23/50
16/16 [==============================] - 0s 875us/step - loss: 0.0316 - accuracy: 0.9959
Epoch 24/50
16/16 [==============================] - 0s 1ms/step - loss: 0.0292 - accuracy: 0.9959
Epoch 25/50
16/16 [==============================] - 0s 1000us/step - loss: 0.0272 - accuracy: 0.9938
Epoch 26/50
16/16 [==============================] - 0s 937us/step - loss: 0.0245 - accuracy: 0.9959
Epoch 27/50
16/16 [==============================] - 0s 937us/step - loss: 0.0218 - accuracy: 0.9959
Epoch 28/50
16/16 [==============================] - 0s 812us/step - loss: 0.0200 - accuracy: 0.9959
Epoc

0.9202454090118408

In [ ]:
# Export our model to HDF5 file
nn.save("trained_por_g3.h5")

# Import the model to a new object
#nn_imported = tf.keras.models.load_model('trained_attrition.h5')

# Evaluate the model using the test data
#model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
#print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [6]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model("trained_por_G2.h5")

In [7]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Create a DataFrame containing training history
history_df = pd.DataFrame(fit_model.history, index=range(1,len(fit_model.history["loss"])+1))

# Plot the loss
display(history_df.plot(y="loss"))

# Plot the accuracy
display(history_df.plot(y="accuracy"))

NameError: name 'X_test_scaled' is not defined

# Conclusion
###### This actually surprised me how high the accuracy is on these models. I thought there would not be nearly enough rows for it to learn from. 
###### On the other hand it is less surprising that the models for the second and third trimesters are so good, because whether or not they passed the previous trimester is a really good predictor. 